## This notebook is used to generate the finalized version of the classifier, to simply feature transformation into the final form, and to test that the results are the same

Most of the code comes from operational_classifier.

In [188]:
import pandas as pd
import numpy as np
import pickle
import sys
#reload(sys)
#sys.setdefaultencoding("utf-8")

#Loading raw data
df = pd.read_csv("../data/labeled_data.csv")
tweets = df.tweet

## Feature generation

In [190]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
import string
import re

stopwords=stopwords = nltk.corpus.stopwords.words("english")

other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)

stemmer = PorterStemmer()


def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    #parsed_text = parsed_text.code("utf-8", errors='ignore')
    return parsed_text

def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]+", tweet.lower())).strip()
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""
    tweet = " ".join(re.split("[^a-zA-Z.,!?]+", tweet.lower())).strip()
    return tweet.split()

vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=tokenize,
    preprocessor=preprocess,
    ngram_range=(1, 3),
    stop_words=stopwords, #We do better when we keep stopwords
    use_idf=True,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    max_features=10000,
    min_df=5,
    max_df=0.501
    )

In [191]:
#Construct tfidf matrix and get relevant scores
tfidf = vectorizer.fit_transform(tweets).toarray()
vocab = {v:i for i, v in enumerate(vectorizer.get_feature_names_out())}
idf_vals = vectorizer.idf_
idf_dict = {i:idf_vals[i] for i in vocab.values()} #keys are indices; values are IDF scores

c:\Users\jxpan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\jxpan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(


In [192]:
#Get POS tags for tweets and save as a string
tweet_tags = []
for t in tweets:
    tokens = basic_tokenize(preprocess(t))
    tags = nltk.pos_tag(tokens)
    tag_list = [x[1] for x in tags]
    #for i in range(0, len(tokens)):
    tag_str = " ".join(tag_list)
    tweet_tags.append(tag_str)
        #print(tokens[i],tag_list[i])

In [193]:
#We can use the TFIDF vectorizer to get a token matrix for the POS tags
pos_vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=None,
    lowercase=False,
    preprocessor=None,
    ngram_range=(1, 3),
    stop_words=None, #We do better when we keep stopwords
    use_idf=False,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    max_features=5000,
    min_df=5,
    max_df=0.501,
    )

In [194]:
#Construct POS TF matrix and get vocab dict
pos = pos_vectorizer.fit_transform(pd.Series(tweet_tags)).toarray()
pos_vocab = {v:i for i, v in enumerate(pos_vectorizer.get_feature_names_out())}

In [196]:
#Now get other features
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
from textstat.textstat import *

sentiment_analyzer = VS()

def count_twitter_objs(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned.
    
    Returns counts of urls, mentions, and hashtags.
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

def other_features(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    ##SENTIMENT
    sentiment = sentiment_analyzer.polarity_scores(tweet)
    
    words = preprocess(tweet) #Get text only
    
    syllables = textstat.syllable_count(words) #count syllables in words
    num_chars = sum(len(w) for w in words) #num chars in words
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
    num_unique_terms = len(set(words.split()))
    
    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59,1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)),2)
    
    twitter_objs = count_twitter_objs(tweet) #Count #, @, and http://
    retweet = 0
    if "rt" in words:
        retweet = 1
    features = [FKRA, FRE,syllables, avg_syl, num_chars, num_chars_total, num_terms, num_words,
                num_unique_terms, sentiment['neg'], sentiment['pos'], sentiment['neu'], sentiment['compound'],
                twitter_objs[2], twitter_objs[1],
                twitter_objs[0], retweet]
    #features = pandas.DataFrame(features)
    return features

def get_feature_array(tweets):
    feats=[]
    for t in tweets:
        feats.append(other_features(t))
    return np.array(feats)

In [197]:
other_features_names = ["FKRA", "FRE","num_syllables", "avg_syl_per_word", "num_chars", "num_chars_total", \
                        "num_terms", "num_words", "num_unique_words", "vader neg","vader pos","vader neu", "vader compound", \
                        "num_hashtags", "num_mentions", "num_urls", "is_retweet"]

In [198]:
feats = get_feature_array(tweets)

In [199]:
#Now join them all up
M = np.concatenate([tfidf,pos,feats],axis=1)

In [200]:
M.shape

(24783, 11166)

In [201]:
#Finally get a list of variable names
variables = ['']*len(vocab)
for k,v in vocab.items():
    variables[v] = k

pos_variables = ['']*len(pos_vocab)
for k,v in pos_vocab.items():
    pos_variables[v] = k

feature_names = variables+pos_variables+other_features_names

# Running the model

This model was found using a GridSearch with 5-fold cross validation. Details are in the notebook operational_classifier.

In [16]:
X = pd.DataFrame(M)
y = df['class'].astype(int)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

In [123]:
select = SelectFromModel(LogisticRegression(class_weight='balanced',penalty="l1",C=0.01,solver='liblinear'))
X_ = select.fit_transform(X,y)

In [186]:
#model = LinearSVC(class_weight='balanced',C=0.01, penalty='l2', loss='squared_hinge',multi_class='ovr', max_iter=10000).fit(X_, y)
model = LogisticRegression(class_weight='balanced',penalty='l2',C=0.01,solver='liblinear').fit(X_,y)

In [125]:
y_preds = model.predict(X_)

In [126]:
report = classification_report( y, y_preds )

In [127]:
print(report)

              precision    recall  f1-score   support

           0       0.44      0.60      0.50      1430
           1       0.97      0.91      0.94     19190
           2       0.82      0.96      0.88      4163

    accuracy                           0.90     24783
   macro avg       0.74      0.82      0.77     24783
weighted avg       0.91      0.90      0.90     24783



# Using information from the model to obtain the matrix X_ generically

This is the most difficult task: We have to take the inputs tweets and transform them into a format that can be used in the model without going through all the same pre-processing steps as above. This can be done as follows.

## Obtaining information about the model

In [202]:
final_features = select.get_support(indices=True) #get indices of features
final_feature_list = [str(feature_names[i]) for i in final_features] #Get list of names corresponding to indices

In [203]:
print(final_feature_list)

['america', 'american', 'anoth', 'ass', 'ass cracker', 'ass hoe', 'ass nigga', 'bad', 'beaner', 'big', 'bird', 'bitch', 'bitch nigga', 'black', 'border', 'born', 'bout', 'browni', 'busi', 'campu', 'charli', 'chink', 'color', 'color folk', 'coon', 'countri', 'cracker', 'crazi', 'crippl', 'cunt', 'da', 'damn', 'darki', 'dick', 'die', 'doe', 'dyke', 'fag', 'faggot', 'fat', 'femal', 'feminist', 'filth', 'first', 'folk', 'fucc nicca', 'fuck', 'fuckin', 'game', 'gay', 'get', 'ghetto', 'girl', 'gon', 'good', 'gook', 'got nigga', 'gt gt', 'hate', 'hate hoe', 'hi', 'hire', 'ho', 'hoe', 'hood', 'hope', 'human', 'israel', 'jap', 'jew', 'jihadi', 'kill', 'lame', 'latina', 'let', 'like', 'lol', 'look like', 'love', 'may', 'mexican', 'mock', 'money', 'monkey', 'much', 'muslim', 'muzzi', 'negro', 'nicca', 'nig', 'nigga', 'nigga bitch', 'niggah', 'niggaz', 'nigger', 'nigguh', 'niglet', 'oreo', 'peopl', 'play', 'pussi', 'queer', 'race', 'racist', 'real', 'real nigga', 'redneck', 'retard', 'sex', 'shit'

In [204]:
#Getting names for each class of features
ngram_features = final_feature_list[:final_feature_list.index('yr')+2]
pos_features = final_feature_list[final_feature_list.index('yr')+2:final_feature_list.index('VBD')+1]
oth_features = final_feature_list[final_feature_list.index('VBD')+1:]

## Generating ngram features

In [205]:
new_vocab = {v:i for i, v in enumerate(ngram_features)}
new_vocab_to_index = {}
for k in ngram_features:
    new_vocab_to_index[k] = vocab[k]

In [206]:
#Get indices of text features
ngram_indices = final_features[:len(ngram_features)]

In [ ]:
#TODO: Pickle new vectorizer

In [208]:
new_vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=tokenize,
    preprocessor=preprocess,
    ngram_range=(1, 3),
    stop_words=stopwords, #We do better when we keep stopwords
    use_idf=False,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    min_df=1,
    max_df=1.0,
    vocabulary=new_vocab
    )

In [209]:
# from sklearn.externals import joblib
# joblib.dump(new_vectorizer, 'final_tfidf.pkl') 
# Saving model to pickle file
with open("final_tfidf_new.pkl", "wb") as file: 
    pickle.dump(new_vectorizer, file) 

In [210]:
tfidf_ = new_vectorizer.fit_transform(tweets).toarray()

In [85]:
#Verifying that results are the same

In [211]:
tfidf_[1,:]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])

In [212]:
tfidf_[1,:].sum()

2.0

In [213]:
X_[1,:tfidf_.shape[1]]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 4.80981477, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 2.81738461, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [214]:
X_[1,:tfidf_.shape[1]].sum()

7.627199378751962

Results are the same if use IDF but the problem is that IDF will be different if we use different data. Instead we have to use the original IDF scores and multiply them by the new matrix.

In [215]:
idf_vals_ = idf_vals[ngram_indices]

In [216]:
idf_vals_.shape

(157,)

In [217]:
#TODO: Pickle idf_vals

#joblib.dump(idf_vals_, 'final_idf.pkl') 
with open("final_idf_new.pkl", "wb") as file: 
    pickle.dump(idf_vals_, file) 

In [219]:
#(tfidf_[1,:]*idf_vals_) == X_[1,:153] #Got same value as final process array!
np.array_equal(tfidf_[1,:]*idf_vals_, X_[1,:157])


True

In [220]:
#tfidf_*idf_vals_ == X_[:,:153]
np.array_equal(tfidf_*idf_vals_, X_[:,:157])


True

In [222]:
tfidffinal = tfidf_*idf_vals_

## Generating POS features
This is simpler as we do not need to worry about IDF but it will be slower as we have to compute the POS tags for the new data. Here we can simply use the old POS tags.

In [223]:
new_pos = {v:i for i, v in enumerate(pos_features)}

In [224]:
#TODO: Pickle pos vectorizer
#We can use the TFIDF vectorizer to get a token matrix for the POS tags
new_pos_vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=None,
    lowercase=False,
    preprocessor=None,
    ngram_range=(1, 3),
    stop_words=None, #We do better when we keep stopwords
    use_idf=False,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    min_df=1,
    max_df=1.0,
    vocabulary=new_pos
    )

In [225]:
#joblib.dump(new_pos_vectorizer, 'final_pos.pkl') 
with open("final_pos_new.pkl", "wb") as file: 
    pickle.dump(new_pos_vectorizer, file) 

In [227]:
pos_ = new_pos_vectorizer.fit_transform(tweet_tags).toarray()

In [228]:
pos_[1,:]

array([1., 1., 1., 1., 0., 0., 0., 0.])

In [229]:
X_[1,157:165]

array([1., 1., 1., 1., 0., 0., 0., 0.])

In [231]:
#pos_[:,:] == X_[:,153:159]
np.array_equal(pos_[:,:], X_[:,157:165])


True

In [232]:
pos_[:,:].sum()

95691.0

In [233]:
X_[:,157:165].sum()

95691.0

## Finally, we can look at the other features

In [234]:
print(other_features_names)

['FKRA', 'FRE', 'num_syllables', 'avg_syl_per_word', 'num_chars', 'num_chars_total', 'num_terms', 'num_words', 'num_unique_words', 'vader neg', 'vader pos', 'vader neu', 'vader compound', 'num_hashtags', 'num_mentions', 'num_urls', 'is_retweet']


In [235]:
print(oth_features)

['FKRA', 'FRE', 'num_syllables', 'num_chars', 'num_chars_total', 'num_words', 'num_unique_words', 'vader compound', 'num_hashtags', 'num_mentions']


The functions can be modified to only calculate and return necessary fields.

In [242]:
def other_features_(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    ##SENTIMENT
    sentiment = sentiment_analyzer.polarity_scores(tweet)
    
    words = preprocess(tweet) #Get text only
    
    syllables = textstat.syllable_count(words) #count syllables in words
    num_chars = sum(len(w) for w in words) #num chars in words
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
    num_unique_terms = len(set(words.split()))
    
    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59,1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)),2)
    
    twitter_objs = count_twitter_objs(tweet) #Count #, @, and http://
    # features = [FKRA, FRE, syllables, num_chars, num_chars_total, num_terms, num_words,
    #             num_unique_terms, sentiment['compound'],
    #             twitter_objs[2], twitter_objs[1],]
    features = [FKRA, FRE, syllables, num_chars, num_chars_total, num_words,
                num_unique_terms, sentiment['compound'],
                twitter_objs[2], twitter_objs[1],]
    #features = pandas.DataFrame(features)
    return features

def get_feature_array_(tweets):
    feats=[]
    for t in tweets:
        feats.append(other_features_(t))
    return np.array(feats)

In [243]:
feats_ = get_feature_array_(tweets)

In [244]:
feats_[0,:]

array([  7.4   ,  86.71  ,  28.    , 127.    , 140.    ,  25.    ,
        23.    ,   0.4563,   0.    ,   1.    ])

In [245]:
X_[0,165:]

array([  7.4   ,  86.71  ,  28.    , 127.    , 140.    ,  25.    ,
        23.    ,   0.4563,   0.    ,   1.    ])

In [246]:
#feats_[:,:] == X_[:,159:]
np.array_equal(feats_[:,:], X_[:,165:])

True

## Now that we have put it all together using a simplified process we can assess if these new data return the same answers.

In [247]:
M_ = np.concatenate([tfidffinal, pos_, feats_],axis=1)

In [248]:
M_.shape

(24783, 175)

In [249]:
X__ = pd.DataFrame(M_)

In [250]:
y_preds_ = model.predict(X__)

In [251]:
report = classification_report( y, y_preds_ )

In [252]:
print(report)

              precision    recall  f1-score   support

           0       0.44      0.60      0.50      1430
           1       0.97      0.91      0.94     19190
           2       0.82      0.96      0.88      4163

    accuracy                           0.90     24783
   macro avg       0.74      0.82      0.77     24783
weighted avg       0.91      0.90      0.90     24783



OK. So now that we have verified that the results are the same with X_ and X__ we can implement a script that can transform new data in this manner.

In [253]:
with open("final_model_new.pkl", "wb") as file: 
    pickle.dump(model, file) 